# Source Detection Benchmark

In [ ]:
from scipy.spatial.distance import cdist
import numpy as np

## Evaluation Measures

true positives (TP), false positives (FP) or false negatives (FN)
true sources correctly detected (TSD) = recall = TP / (TP + FN)
reliability = precision = TP / (TP + FP)

The strategy used to match the detections with the sources in the reference catalogue is the following: first, the closest detection for each source in the catalogue is found. Second, associations between sources and detections that are further away than a pre-established maximum distance are deleted. We assume that the centre of two different sources cannot be closer than the FWHM of the image, so this is the maximum distance selected. Next, where detections associated with more than one source from the catalogue are found, only the closest association is kept, deleting the others. In the same way, if a true source is associated with more than one detection, only the correspondence with the shortest distance is saved. Finally, the evaluation measures are computed: TP are the detections associated with a source, FP are detections without any associated source, while FN are sources with no associations with a detection (AUTOMATIC SOURCE DETECTION IN ASTRONOMICAL IMAGES, Marc MASIAS MOYSET 2014).

In [ ]:
def automatic_assignment(gtruth, predicted, max_dist):
    eucl_dists = cdist(gtruth, predicted)
    alloc_idxs = np.argmin(eucl_dists, axis=1) # pics first if multiple sources would have exaclty same min distance
    min_eucl_dists = np.array([eucl_dists[i,j] for i, j in zip(np.arange(eucl_dists.shape[0]), alloc_idxs)])
    assignment_idxs = np.where(min_eucl_dists <= max_dist)[0]
    # marks the gtruth which have a valid prediction assignment as True
    gtruth_predicted = np.array([False] * gtruth.shape[0])
    gtruth_predicted[assignment_idxs] = True
    # marks the predictions with a valid assignment of the gtruth as True
    predicted_assigned = np.array([False] * predicted.shape[0])
    predicted_assigned[alloc_idxs[assignment_idxs]] = True
    return gtruth_predicted, predicted_assigned

def calc_eval_measures(gtruth, predicted, max_dist):
    gtruth_predicted, predicted_assigned = automatic_assignment(gtruth, predicted, max_dist)
    fn = len(gtruth_predicted) - np.sum(gtruth_predicted)
    tp = np.sum(predicted_assigned)
    fp = len(predicted_assigned) - tp
    return tp, fp, fn

In [ ]:
# with this seed and max_dist it seems to be correct 
# to validate compare indices of output of automatig_assignment marked as True to below  set indices
np.random.seed(42)
max_dist = 10
gtruth = np.random.rand(7,2)*2048
predicted = np.random.rand(8,2)*2048
predicted[[2,3,6,7,0]] = gtruth[[4,2,5,0,1]]
calc_eval_measures(gtruth, predicted, max_dist)